In [464]:
import requests 

# # This doesn't work as need to include GitHub authentication for AdventOfCode site
# sonar_url = 'https://adventofcode.com/2021/day/1/input'
# depths = requests.get(sonar_url).text

# Using text file and looping through a list

In [168]:
def depth_input(input_file:str):
    """Build array from single column text file. Convert to integers"""
    depths = []
    
    with open(input_file, 'r') as file:
        for line in file:
            try:
                depths.append(int(line.strip()))
            except ValueError as e: 
                    pass
                
    return depths

In [461]:
def count_increases(depths:list) -> int:
    """Returns the number of increases between elements in a list """
    increase = 0
    for i in range(len(depths)):
        if i == 0:
            pass
        elif depths[i] > depths[i-1]:
            increase += 1
    return increase


def count_window_increases(depths:list, window:int) -> int:
    """ Reads the number of increases between rolling window 
    """
    increase = 0
    # skipping first two elements as can't form full 3 element windows - using subraction on index
    for i in range(window, (len(depths))):
        current_window = sum(depths[i-window:i])
        next_window = sum(depths[i-window+1:i+1])
        if current_window < next_window:
            increase += 1
    return increase

In [465]:
depths = depth_input('data/sonar_depths.txt')
increases = count_increases(depths)
window_increases = count_window_increases(depths,3)

print(f'Number of increases between readings: {increases}')
print(f'Number of increases between rolling windows: {window_increases}')

Number of increases between readings: 1482
Number of increases between rolling windows: 1518


## Using NumPy rather than list and loops

In [466]:
import numpy as np

def np_input(input_file:str) -> np.array: 
    """Build array of integers from single column text file. Load through numpy"""
    with open(input_file, 'r') as file:
        depths = np.loadtxt(file, dtype=np.int16)
    return depths


def np_change(depths:np.array) -> np.array:
    """
    Returns the difference between an element and the following in an array.  Length is one less than input 
    """
    change = depths[1:] - depths[:-1]
    return change

In [467]:
def np_3_window(depths:np.array) -> np.array:
    """Static window of three elements."""
    summed_window = depths[:-2] + depths[1:-1] + depths[2:]
    return summed_window


def np_window(depths:np.array, window:int) -> np.array:
    """
    Returns a sum of all elements using a window size.
    Loops through size of window and sums resulting array slices.  
    If selecting a window of 1 it will return the original array.
    
    Note: needs to copy slice otherwise adds to original array
    """
    # First element up to window length
    end_idx = (1-window)   if not window == 1 else None
    summed_window = np.copy(depths[:end_idx])
    # Add each subsequent elements for the window sum
    for i in range(1, window):
        start_idx = (0 + i)
        end_idx =   (1 + i - window)   if not (i - window == -1) else None
        summed_window += depths[start_idx:end_idx]
    return summed_window

In [468]:
def sum_of_increases(array:np.array) -> int:
    """Returns the number of positive values in an array"""
    return sum(array > 0)

In [469]:
np_depths = np_input('data/sonar_depths.txt')

single_increases = sum_of_increases(np_change(np_depths))

window_increases = sum_of_increases(np_change(np_window(np_depths,3)))


print(f'Number of increases between readings: {single_increases}')
print(f'Number of increases between rolling 3 element windows: {window_increases}')

Number of increases between readings: 1482
Number of increases between rolling 3 element windows: 1518
